In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [13]:
EMIS = pd.read_csv("EMIS Patient postcodes.csv")
# InfoFlex = pd.read_csv("Postcodes 2022 - Mar 2023.txt")

In [14]:
# print(InfoFlex.head())
print(EMIS.head())

   EMIS Number Postcode Lower Layer Area (2011) Middle Layer Area (2011)  \
0          1.0  DA7 5AA               E01000327                E02000081   
1          3.0  DA3 7PJ               E01024158                E02005040   
2          4.0  DA1 1XD               E01024162                E02005032   
3          5.0  DA2 6EG               E01024138                E02005035   
4          6.0  DA2 6EG               E01024138                E02005035   

   Unnamed: 4  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  


In [15]:
postcodes_emis = EMIS['Postcode'].dropna()
# postcodes_infoflex = InfoFlex['Postcode'].dropna()
# all_postcodes = pd.concat([postcodes_emis, postcodes_infoflex]).drop_duplicates().reset_index(drop=True)
all_postcodes = postcodes_emis.drop_duplicates().reset_index(drop=True)

Postcodes = pd.DataFrame({'Postcode': all_postcodes})

In [16]:
print(Postcodes.nunique())
print(Postcodes.info())
print(Postcodes.head())

Postcode    2227
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2227 entries, 0 to 2226
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Postcode  2227 non-null   object
dtypes: object(1)
memory usage: 17.5+ KB
None
   Postcode
0   DA7 5AA
1   DA3 7PJ
2   DA1 1XD
3   DA2 6EG
4  DA11 9DL


In [17]:
coord_df, map_obj = create_postcode_map(Postcodes, 'Postcode')
analyze_coverage(coord_df)

Processing 2227 postcodes...
Processed 0/2227 postcodes...


Processed 50/2227 postcodes...
Failed to get coordinates for ME2 2FN
Processed 100/2227 postcodes...
Processed 150/2227 postcodes...
Processed 200/2227 postcodes...
Processed 250/2227 postcodes...
Failed to get coordinates for DA1 1HX
Failed to get coordinates for DA7 9QL
Processed 300/2227 postcodes...
Processed 350/2227 postcodes...
Failed to get coordinates for DA12 AAQ
Processed 400/2227 postcodes...
Processed 450/2227 postcodes...
Processed 500/2227 postcodes...
Processed 550/2227 postcodes...
Failed to get coordinates for DA12 0UN
Processed 600/2227 postcodes...
Processed 650/2227 postcodes...
Processed 700/2227 postcodes...
Processed 750/2227 postcodes...
Processed 800/2227 postcodes...
Processed 850/2227 postcodes...
Failed to get coordinates for BR9 7TW
Processed 900/2227 postcodes...
Failed to get coordinates for DA12 3ET
Processed 950/2227 postcodes...
Failed to get coordinates for ME5 5EG
Processed 1000/2227 postcodes...
Processed 1050/2227 postcodes...
Processed 1100/2227 

In [18]:
import pandas as pd
import requests
import folium
from folium import plugins
import time
import json

def get_postcode_coordinates(postcode):
    """
    Get latitude and longitude for a UK postcode using the free postcodes.io API
    """
    try:
        # Clean the postcode (remove extra spaces)
        postcode = postcode.strip().upper()
        
        # Make API request
        response = requests.get(f"https://api.postcodes.io/postcodes/{postcode}")
        
        if response.status_code == 200:
            data = response.json()
            if data['status'] == 200:
                result = data['result']
                return {
                    'postcode': postcode,
                    'latitude': result['latitude'],
                    'longitude': result['longitude'],
                    'admin_district': result.get('admin_district', ''),
                    'admin_county': result.get('admin_county', ''),
                    'country': result.get('country', '')
                }
        
        print(f"Failed to get coordinates for {postcode}")
        return None
        
    except Exception as e:
        print(f"Error processing {postcode}: {str(e)}")
        return None

def create_postcode_map(df, postcode_column='postcode'):
    """
    Create an interactive map showing all postcodes from the dataframe
    
    Parameters:
    df: pandas DataFrame containing postcodes
    postcode_column: name of the column containing postcodes
    """
    
    print(f"Processing {len(df)} postcodes...")
    
    # Get coordinates for all postcodes
    coordinates = []
    failed_postcodes = []
    
    for i, postcode in enumerate(df[postcode_column].unique()):
        if i % 50 == 0:  # Progress indicator
            print(f"Processed {i}/{len(df)} postcodes...")
        
        coord = get_postcode_coordinates(postcode)
        if coord:
            coordinates.append(coord)
        else:
            failed_postcodes.append(postcode)
        
        # Be nice to the API - small delay between requests
        time.sleep(0.1)
    
    print(f"\nSuccessfully geocoded {len(coordinates)} postcodes")
    if failed_postcodes:
        print(f"Failed to geocode {len(failed_postcodes)} postcodes: {failed_postcodes[:10]}...")
    
    if not coordinates:
        print("No coordinates found! Check your postcodes.")
        return None
    
    # Create DataFrame with coordinates
    coord_df = pd.DataFrame(coordinates)
    
    # Calculate center of map (mean of all coordinates)
    center_lat = coord_df['latitude'].mean()
    center_lon = coord_df['longitude'].mean()
    
    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=6,
        tiles='OpenStreetMap'
    )
    
    # Add markers for each postcode
    for idx, row in coord_df.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            popup=f"""
            <b>{row['postcode']}</b><br>
            District: {row['admin_district']}<br>
            County: {row['admin_county']}<br>
            Country: {row['country']}
            """,
            tooltip=row['postcode'],
            color='red',
            fillColor='red',
            fillOpacity=0.7
        ).add_to(m)
    
    # Add a marker cluster option (uncomment if you have many postcodes)
    # marker_cluster = plugins.MarkerCluster().add_to(m)
    # for idx, row in coord_df.iterrows():
    #     folium.Marker(
    #         location=[row['latitude'], row['longitude']],
    #         popup=f"{row['postcode']}<br>{row['admin_district']}",
    #         tooltip=row['postcode']
    #     ).add_to(marker_cluster)
    
    # Add a heatmap layer (optional)
    heat_data = [[row['latitude'], row['longitude']] for idx, row in coord_df.iterrows()]
    plugins.HeatMap(heat_data, radius=15, blur=10, gradient={0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1.0: 'red'}).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Save the map
    map_filename = 'uk_postcode_map.html'
    m.save(map_filename)
    print(f"\nMap saved as '{map_filename}'")
    print(f"Open this file in your web browser to view the interactive map")
    
    # Return the coordinate data for further analysis
    return coord_df, m

def analyze_coverage(coord_df):
    """
    Provide some basic analysis of the postcode coverage
    """
    if coord_df is None or coord_df.empty:
        return
    
    print("\n=== COVERAGE ANALYSIS ===")
    print(f"Total postcodes mapped: {len(coord_df)}")
    
    # Group by administrative areas
    if 'admin_district' in coord_df.columns:
        district_counts = coord_df['admin_district'].value_counts()
        print(f"\nTop 10 districts by postcode count:")
        print(district_counts.head(10))
    
    if 'admin_county' in coord_df.columns:
        county_counts = coord_df['admin_county'].value_counts()
        print(f"\nTop 10 counties by postcode count:")
        print(county_counts.head(10))
    
    if 'country' in coord_df.columns:
        country_counts = coord_df['country'].value_counts()
        print(f"\nCountries covered:")
        print(country_counts)

print("Code loaded! To create your map, run:")
print("coord_df, map_obj = create_postcode_map(df, 'postcode')")
print("analyze_coverage(coord_df)")
print("\nMake sure to replace 'df' and 'postcode' with your actual dataframe and column names.")

Code loaded! To create your map, run:
coord_df, map_obj = create_postcode_map(df, 'postcode')
analyze_coverage(coord_df)

Make sure to replace 'df' and 'postcode' with your actual dataframe and column names.
